In [421]:
# import 
import json
import sys

import pandas

In [422]:
# setting

## columns
COLUMNS_CALCED = [
    'average_all', 'average', 'average_exclude_debut', 'highest', 'highest_exclude_debut',
    'over30', 'over50', 'over100', 'over200', 'under30', 'under50', 'under100', 'under200', 'count_all', 'count'
]
COLUMNS = [
    'title', 'date', 'initial', 'kind', # 'r2019u',
    'r2007', 'r2008', 'r2009', 'r2010', 'r2011', 'r2012', 'r2013', 'r2014', 'r2015', 'r2016', 'r2017', 'r2018', 'r2019',
] + COLUMNS_CALCED
RECENT = 'r2019'
BEFORE = 'r2018'
URA = 'r2019'
RANK_ALL = ['r2005', 'r2006#1', 'r2006#2', 'r2007', 'r2008', 'r2009', 'r2010', 'r2011', 'r2012', 'r2013', 'r2014', 'r2015', 'r2016', 'r2017', 'r2018', 'r2019']
RANK = RANK_ALL[3:]

def year_filter(value: str, initial: str, array: list, mode: str = 'ge'):
    if mode not in ['gt', 'ge']:
        raise ValueError(f'invalid mode: {mode}')
    if value in array and initial in array:
        if mode == 'ge':
            return array.index(value) >= array.index(initial)
        else:
            return array.index(value) > array.index(initial)
    elif value in array:
        return True
    else:
        return False

### summary.json

In [423]:
# read json
SUMMARY_PATH = './summary.json'
with open(SUMMARY_PATH, encoding='utf8') as summary_file:
    summary = json.load(summary_file)

In [424]:
# make main dataframe
df = pandas.DataFrame(summary)
df.index = df.no
df['average_all'] = df.average
df = df.drop(columns=['no', 'average'], axis=0)
df.date = pandas.to_datetime(df.date)
df.head()

,title,ruby,inst,hall_of_fame,works,debut,date,r2019u,initial,kind,...,r2011,r2012,r2013,r2014,r2015,r2016,r2017,r2018,r2019,average_all
no,,,,,,,,,,,,,,,,,,,,,
1,桜の花は狂い咲き,None,False,False,"[SOME GIRLS -REBEL STREEL IV, CLASSICS, 月光嗜好症]",SOME GIRLS -REBEL STREEL IV,1997-04-25,64.0,r2005,"[SA, OA, OT]",...,96.0,123.0,68.0,169.0,165.0,100.0,134.0,164.0,123.0,121.462
2,青蛾月,None,False,False,"[幻想庭園, étoiles]",幻想庭園,1988-01-25,22.0,r2005,"[SA, OA]",...,72.0,57.0,54.0,87.0,81.0,59.0,146.0,126.0,54.0,80.923
3,マリーゴールド・ガーデン,None,False,False,"[幻想庭園, フラワーチャイルド, 神々の黄昏]",幻想庭園,1988-01-25,128.0,r2005,"[SA, OA, S-2]",...,175.0,141.0,218.0,141.0,238.0,190.0,155.0,164.0,216.0,171.769
4,鏡面界 im Juni,きょうめんかい・イン・ジュニ,False,False,[幻想庭園],幻想庭園,1988-01-25,150.0,r2005,[OA],...,204.0,131.0,174.0,194.0,253.0,244.0,202.0,250.0,260.0,201.077
5,アンジェ・ノワールの祭戯,None,False,False,"[幻想庭園, 神々の黄昏]",幻想庭園,1988-01-25,NaN,r2005,"[SA, OA]",...,81.0,96.0,150.0,106.0,156.0,244.0,187.0,63.0,106.0,125.615


#### works.json

In [425]:
WORKS_PATH = './works.json'
with open(WORKS_PATH, encoding='utf8') as works_file:
    works = json.load(works_file)

In [426]:
dfw = pandas.DataFrame(works)
dfw.date = pandas.to_datetime(dfw.date)
dfw.head()

,title,kind,date
0,SOME GIRLS -REBEL STREEL IV,OT,1997-04-25
1,幻想庭園,OA,1988-01-25
2,フラワーチャイルド,S,1988-07-25
3,恋せよ乙女 -Love story of ZIPANG-,S,1992-07-07
4,月下の一群,OA,1992-12-09


#### info.json

In [427]:
INFO_PATH = './json/info.json'
with open(INFO_PATH, encoding='utf8') as info_file:
    info = json.load(info_file)

lowest = info['max']

* average  
平均順位  
注) データが揃っている2007年以降
* average_exclude_debut  
デビュー年以外の平均順位
* highest  
最高順位
* highest_exclude_debut  
デビュー年以外の最高順位
* over30  
30位を超えた回数
* over50  
50位を超えた回数
* over100  
100位を超えた回数
* over200  
200いを超えた回数
* count_all  
登場回数
* count  
データが揃っている登場回数

In [428]:
df['average'] = 0.
df['average_exclude_debut'] = 0.
df['highest'] = 0.
df['highest_exclude_debut'] = 0.
df['lowest'] = 0.
df['over30'] = 0
df['over50'] = 0
df['over100'] = 0
df['over200'] = 0
df['under30'] = 0
df['under50'] = 0
df['under100'] = 0
df['under200'] = 0
df['count_all'] = 0
df['count'] = 0

for no in df.index:
    song = df.loc[no]
    to_mean = list(filter(lambda y: year_filter(y, song.initial, RANK), RANK))
    to_mean_exclude_debut = list(filter(lambda y: year_filter(y, song.initial, RANK, 'gt'), RANK))
    df.average.at[no] = song[to_mean].mean()
    df.average_exclude_debut.at[no] = song[to_mean_exclude_debut].mean()
    df.highest_exclude_debut.at[no] = song[to_mean_exclude_debut].min()
    to_mean_all = list(filter(lambda y: year_filter(y, song.initial, RANK_ALL), RANK_ALL))
    df.highest.at[no] = song[to_mean_all].min()
    df.lowest.at[no] = song[to_mean_all].max()
    df.over30.at[no] = len(list(filter(lambda s: s > 30, song[to_mean_all].values)))
    df.over50.at[no] = len(list(filter(lambda s: s > 50, song[to_mean_all].values)))
    df.over100.at[no] = len(list(filter(lambda s: s > 100, song[to_mean_all].values)))
    df.over200.at[no] = len(list(filter(lambda s: s > 200, song[to_mean_all].values)))
    df.under30.at[no] = len(list(filter(lambda s: s <= 30, song[to_mean_all].values)))
    df.under50.at[no] = len(list(filter(lambda s: s <= 50, song[to_mean_all].values)))
    df.under100.at[no] = len(list(filter(lambda s: s <= 100, song[to_mean_all].values)))
    df.under200.at[no] = len(list(filter(lambda s: s <= 200, song[to_mean_all].values)))
    df.count_all.at[no] = len(to_mean_all)
    df.at[no, 'count'] = len(to_mean)

df.head()
# df.iloc[145]

,title,ruby,inst,hall_of_fame,works,debut,date,r2019u,initial,kind,...,over30,over50,over100,over200,under30,under50,under100,under200,count_all,count
no,,,,,,,,,,,,,,,,,,,,,
1,桜の花は狂い咲き,None,False,False,"[SOME GIRLS -REBEL STREEL IV, CLASSICS, 月光嗜好症]",SOME GIRLS -REBEL STREEL IV,1997-04-25,64.0,r2005,"[SA, OA, OT]",...,13,13,9,0,0,0,4,13,16,13
2,青蛾月,None,False,False,"[幻想庭園, étoiles]",幻想庭園,1988-01-25,22.0,r2005,"[SA, OA]",...,13,13,2,0,0,0,11,13,16,13
3,マリーゴールド・ガーデン,None,False,False,"[幻想庭園, フラワーチャイルド, 神々の黄昏]",幻想庭園,1988-01-25,128.0,r2005,"[SA, OA, S-2]",...,13,13,13,3,0,0,0,10,16,13
4,鏡面界 im Juni,きょうめんかい・イン・ジュニ,False,False,[幻想庭園],幻想庭園,1988-01-25,150.0,r2005,[OA],...,13,13,13,6,0,0,0,7,16,13
5,アンジェ・ノワールの祭戯,None,False,False,"[幻想庭園, 神々の黄昏]",幻想庭園,1988-01-25,NaN,r2005,"[SA, OA]",...,13,13,9,1,0,0,4,12,16,13


## Hall of Fame

In [480]:
df.query('hall_of_fame')[['title', *RANK_ALL]]

,title,r2005,r2006#1,r2006#2,r2007,r2008,r2009,r2010,r2011,r2012,r2013,r2014,r2015,r2016,r2017,r2018,r2019
no,,,,,,,,,,,,,,,,,
125,愛と誠,4.0,5.0,8.0,5.0,6.0,4.0,2.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
143,亡國覚醒カタルシス,NaN,NaN,3.0,2.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
279,リュウコウセカイ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN


In [430]:
almost = df.query('highest == 1 & ~hall_of_fame & initial != @RECENT')[['title', 'date', *RANK_ALL]]
almost['count'] = (almost == 1).sum(1)
almost

,title,date,r2005,r2006#1,r2006#2,r2007,r2008,r2009,r2010,r2011,r2012,r2013,r2014,r2015,r2016,r2017,r2018,r2019,count
no,,,,,,,,,,,,,,,,,,,
99,戦争と平和,2002-07-24,1.0,1.0,10.0,10.0,7.0,11.0,12.0,12.0,7.0,12.0,14.0,25.0,16.0,15.0,45.0,37.0,2
132,北京LOVERS,2005-06-22,NaN,11.0,18.0,16.0,24.0,17.0,20.0,11.0,11.0,9.0,6.0,8.0,11.0,1.0,13.0,7.0,1
135,聖少女領域,2005-10-26,NaN,4.0,4.0,1.0,2.0,7.0,6.0,3.0,5.0,2.0,3.0,3.0,3.0,10.0,2.0,4.0,1
152,勇侠青春謳,2006-10-25,NaN,NaN,1.0,3.0,4.0,10.0,8.0,7.0,28.0,20.0,24.0,24.0,24.0,18.0,39.0,12.0,1
326,芸術変態論,2018-07-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,6.0,1


In [431]:
for no in almost.index:
    print(f'{almost.title.at[no]}|{almost.date.at[no].year}|{almost.at[no, "count"]}')

戦争と平和|2002|2
北京LOVERS|2005|1
聖少女領域|2005|1
勇侠青春謳|2006|1
芸術変態論|2018|1


# MAIN ranking

In [432]:
# target: exclude hall of fame and instrumentals
target = df.query('~hall_of_fame & ~inst').loc[:]
target['rank_average'] = target.average.rank(method='min', na_option='bottom')
target['rank_average_exclude_debut'] = target.average_exclude_debut.rank(method='min', na_option='bottom')
target['rank_highest'] = target.highest.rank(method='min', na_option='bottom')
target['rank_highest_exclude_debut'] = target.highest_exclude_debut.rank(method='min', na_option='bottom')
target.head()

,title,ruby,inst,hall_of_fame,works,debut,date,r2019u,initial,kind,...,under30,under50,under100,under200,count_all,count,rank_average,rank_average_exclude_debut,rank_highest,rank_highest_exclude_debut
no,,,,,,,,,,,,,,,,,,,,,
1,桜の花は狂い咲き,None,False,False,"[SOME GIRLS -REBEL STREEL IV, CLASSICS, 月光嗜好症]",SOME GIRLS -REBEL STREEL IV,1997-04-25,64.0,r2005,"[SA, OA, OT]",...,0,0,4,13,16,13,155.0,133.0,193.0,148.0
2,青蛾月,None,False,False,"[幻想庭園, étoiles]",幻想庭園,1988-01-25,22.0,r2005,"[SA, OA]",...,0,0,11,13,16,13,98.0,77.0,168.0,120.0
3,マリーゴールド・ガーデン,None,False,False,"[幻想庭園, フラワーチャイルド, 神々の黄昏]",幻想庭園,1988-01-25,128.0,r2005,"[SA, OA, S-2]",...,0,0,0,10,16,13,226.0,210.0,283.0,254.0
4,鏡面界 im Juni,きょうめんかい・イン・ジュニ,False,False,[幻想庭園],幻想庭園,1988-01-25,150.0,r2005,[OA],...,0,0,0,7,16,13,276.0,252.0,269.0,242.0
5,アンジェ・ノワールの祭戯,None,False,False,"[幻想庭園, 神々の黄昏]",幻想庭園,1988-01-25,NaN,r2005,"[SA, OA]",...,0,0,4,12,16,13,163.0,140.0,183.0,137.0


In [433]:
# dfw: works
for y in RANK_ALL:
    if y in RANK:
        dfw[f'{y}_average'] = 0.
    dfw[f'{y}_highest'] = 0.

for idx in dfw.index:
    df_tmp = df[df.works.apply(lambda ws: dfw.at[idx, 'title'] in ws)][RANK_ALL]
    mean = df_tmp.mean()
    for i in mean.index:
        if i not in RANK:
            continue
        dfw.at[idx, f'{i}_average'] = mean.at[i]
    high = df_tmp.min()
    for i in high.index:
        dfw.at[idx, f'{i}_highest'] = high.at[i]

dfw.head()

,title,kind,date,r2005_highest,r2006#1_highest,r2006#2_highest,r2007_average,r2007_highest,r2008_average,r2008_highest,...,r2015_average,r2015_highest,r2016_average,r2016_highest,r2017_average,r2017_highest,r2018_average,r2018_highest,r2019_average,r2019_highest
0,SOME GIRLS -REBEL STREEL IV,OT,1997-04-25,NaN,NaN,NaN,86.000000,86.0,126.000000,126.0,...,165.0,165.0,100.0,100.0,134.0,134.0,164.0,164.0,123.0,123.0
1,幻想庭園,OA,1988-01-25,NaN,NaN,NaN,146.090909,84.0,150.181818,77.0,...,178.2,81.0,184.6,59.0,182.9,130.0,157.4,56.0,185.7,54.0
2,フラワーチャイルド,S,1988-07-25,NaN,NaN,NaN,164.000000,141.0,179.500000,162.0,...,245.5,238.0,217.0,190.0,196.0,155.0,160.0,156.0,238.0,216.0
3,恋せよ乙女 -Love story of ZIPANG-,S,1992-07-07,13.0,13.0,NaN,37.000000,36.0,48.500000,48.0,...,62.0,31.0,138.5,52.0,66.5,20.0,41.0,24.0,88.0,44.0
4,月下の一群,OA,1992-12-09,19.0,13.0,NaN,104.000000,36.0,111.300000,48.0,...,166.6,34.0,187.1,104.0,162.8,54.0,177.7,43.0,150.1,52.0


## recent

In [434]:
target_recent = target[['title', 'works', 'debut', 'date', 'initial', 'kind', RECENT, *COLUMNS_CALCED]].astype({RECENT: int}).sort_values(RECENT)

target_recent.head()

,title,works,debut,date,initial,kind,r2019,average_all,average,average_exclude_debut,...,over30,over50,over100,over200,under30,under50,under100,under200,count_all,count
no,,,,,,,,,,,,,,,,,,,,,
338,BArADiPArADicA,[Fantasia],Fantasia,2019-08-28,r2019,[OA],1,1.000,1.000000,NaN,...,0,0,0,0,1,1,1,1,1,1
343,自由戀愛,[Fantasia],Fantasia,2019-08-28,r2019,[OA],2,2.000,2.000000,NaN,...,0,0,0,0,1,1,1,1,1,1
282,Lolicate,"[流行世界, 血と蜜 ～Anthology of Gothic Lolita＆Horror]",流行世界,2014-08-27,r2014,"[OA, CA]",3,6.833,6.833333,7.800000,...,0,0,0,0,6,6,6,6,6,6
135,聖少女領域,"[聖少女領域, ローゼンメイデン・トロイメント オリジナルサウンドトラック, ローゼンメイデ...",聖少女領域,2005-10-26,r2006#1,"[S-1, OT, ST, CA]",4,3.933,3.923077,3.923077,...,0,0,0,0,15,15,15,15,15,13
107,月蝕グランギニョル,"[月蝕グランギニョル, Avenger O.S.T., COLLECTION SIMPLE ...",月蝕グランギニョル,2003-11-05,r2005,"[S-1, SA, ST, CA]",5,4.750,4.846154,4.846154,...,0,0,0,0,16,16,16,16,16,13


In [435]:
for no in target_recent.head(20).index:
    mark = '**' if target_recent.initial.at[no] == RECENT else ''
    print(f'{target_recent.r2019.at[no]}|{mark}{target_recent.title.at[no]}{mark}|{target_recent.date.at[no].year}')

1|**BArADiPArADicA**|2019
2|**自由戀愛**|2019
3|Lolicate|2014
4|聖少女領域|2005
5|月蝕グランギニョル|2003
6|芸術変態論|2018
7|北京LOVERS|2005
8|未來のイヴ|2003
9|雪華懺悔心中|2008
10|GOD DIVA|2006
11|**少女のための残酷童話**|2019
12|勇侠青春謳|2006
13|今宵、碧い森深く|2006
14|**リアル×ファンタジー**|2019
15|**21世紀新青年**|2019
16|暗黒サイケデリック|2007
17|凶夢伝染|2012
18|卑弥呼外伝|2017
19|青嵐血風録|2007
20|人生美味礼讃|2005


In [498]:
# 初登場かどうか
top20 = pandas.DataFrame({r: [False] * 20 for r in RANK}, index=range(1, 21))
for r in RANK:
    tmp = df[df[r] <= 20].sort_values(r)[['title', r, 'initial']]
    for no in tmp.index:
        top20.at[int(tmp.at[no, r]), r] |= tmp.initial.at[no] == r

top20
# top20.sum()

,r2007,r2008,r2009,r2010,r2011,r2012,r2013,r2014,r2015,r2016,r2017,r2018,r2019
1,False,False,False,False,False,False,False,True,False,False,False,True,True
2,False,False,True,False,True,True,False,True,False,True,False,False,True
3,False,False,True,False,False,True,True,False,False,False,False,False,False
4,False,False,False,False,False,False,True,False,False,False,True,False,False
5,False,False,True,False,False,False,True,True,True,False,False,False,False
6,False,False,False,False,False,False,True,False,False,True,False,True,False
7,False,False,False,True,False,False,False,False,True,False,False,False,False
8,False,True,False,False,True,True,False,False,False,False,False,False,False
9,False,True,True,False,True,False,False,False,False,False,False,False,False
10,False,False,False,True,False,False,False,False,False,True,False,True,False


In [437]:
targetw_recent = dfw[['title', 'kind', 'date', f'{RECENT}_average', f'{RECENT}_highest']]

targetw_recent.head()

,title,kind,date,r2019_average,r2019_highest
0,SOME GIRLS -REBEL STREEL IV,OT,1997-04-25,123.0,123.0
1,幻想庭園,OA,1988-01-25,185.7,54.0
2,フラワーチャイルド,S,1988-07-25,238.0,216.0
3,恋せよ乙女 -Love story of ZIPANG-,S,1992-07-07,88.0,44.0
4,月下の一群,OA,1992-12-09,150.1,52.0


### debut

In [438]:
debut = target_recent.query('initial == @RECENT')[['title', RECENT, 'debut', 'date']]
debut

,title,r2019,debut,date
no,,,,
338,BArADiPArADicA,1,Fantasia,2019-08-28
343,自由戀愛,2,Fantasia,2019-08-28
344,少女のための残酷童話,11,Fantasia,2019-08-28
340,リアル×ファンタジー,14,Fantasia,2019-08-28
339,21世紀新青年,15,Fantasia,2019-08-28
341,Maison de Bonbonnière,21,Fantasia,2019-08-28
348,WALTZ FOR DEBBY (Bill Evans),55,Fantasia,2019-08-28
345,恐怖の頭脳改革,61,Fantasia,2019-08-28
337,美しき獣たちの為の,68,『コードギアス 復活のルルーシュ』オリジナルサウンドトラック,2019-02-13


In [439]:
for idx in range(len(debut)):
    print(f'{idx+1}|{debut.iloc[idx].title}|{debut.iloc[idx].debut}')

1|BArADiPArADicA|Fantasia
2|自由戀愛|Fantasia
3|少女のための残酷童話|Fantasia
4|リアル×ファンタジー|Fantasia
5|21世紀新青年|Fantasia
6|Maison de Bonbonnière|Fantasia
7|WALTZ FOR DEBBY (Bill Evans)|Fantasia
8|恐怖の頭脳改革|Fantasia
9|美しき獣たちの為の|『コードギアス 復活のルルーシュ』オリジナルサウンドトラック
10|地球で|Fantasia
11|月下、緑雨幻想|Fantasia


In [440]:
# recent 5 originals
or5 = dfw.query('kind == "OA"').tail(5)
or5['initial'] = or5.date.apply(lambda d: d.year)
initials = sorted(set(or5.initial.values))
col = [*[f'r{y}_average' for y in initials], *[f'r{y}_highest' for y in initials]]
or5[['title', *col]]

,title,r2014_average,r2015_average,r2016_average,r2018_average,r2019_average,r2014_highest,r2015_highest,r2016_highest,r2018_highest,r2019_highest
104,流行世界,19.333333,88.1,107.100000,115.222222,152.777778,1.0,1.0,1.0,8.0,3.0
106,快楽のススメ,NaN,44.6,102.100000,143.400000,151.400000,NaN,5.0,26.0,63.0,53.0
109,A級戒厳令,NaN,NaN,25.777778,89.777778,113.777778,NaN,NaN,2.0,9.0,22.0
114,芸術変態論,NaN,NaN,NaN,30.100000,83.300000,NaN,NaN,NaN,1.0,6.0
116,Fantasia,NaN,NaN,NaN,NaN,41.900000,NaN,NaN,NaN,NaN,1.0


In [441]:
target.groupby(by='initial')['title'].count()

initial
r2005      116
r2006#1      6
r2006#2     15
r2007        4
r2008       18
r2009       16
r2010       22
r2011       25
r2012       13
r2013       13
r2014        9
r2015       20
r2016        9
r2017        4
r2018       10
r2019       11
Name: title, dtype: int64

In [442]:
print(f'total : {len(df):>3d}')
print(f'target: {len(target):>3d}')
print(f'debut : {len(debut):>3d}')

total : 348
target: 311
debut :  11


## total

In [500]:
average_top = target.sort_values(['rank_average_exclude_debut', 'date'], ascending=[True, True]) \
                    .query('average_exclude_debut <= 20') \
                    [['title', 'rank_average_exclude_debut', 'average_exclude_debut', 'date']].astype({'rank_average_exclude_debut': int})

average_top

,title,rank_average_exclude_debut,average_exclude_debut,date
no,,,,
135,聖少女領域,1,3.923077,2005-10-26
107,月蝕グランギニョル,2,4.846154,2003-11-05
191,雪華懺悔心中,3,5.800000,2008-08-27
326,芸術変態論,4,6.000000,2018-07-25
282,Lolicate,5,7.800000,2014-08-27
108,未來のイヴ,6,11.076923,2003-11-05
132,北京LOVERS,7,11.846154,2005-06-22
319,まだら恋椿外道,8,13.000000,2016-08-24
145,GOD DIVA,9,13.538462,2006-06-28


In [444]:
highest_top = target.sort_values(['rank_highest', 'date'], ascending=[True, True]) \
                    .query('highest <= 20') \
                    [['title', 'rank_highest', 'highest', 'date']].astype({'rank_highest': int, 'highest': int})

highest_top

,title,rank_highest,highest,date
no,,,,
99,戦争と平和,1,1,2002-07-24
132,北京LOVERS,1,1,2005-06-22
135,聖少女領域,1,1,2005-10-26
152,勇侠青春謳,1,1,2006-10-25
326,芸術変態論,1,1,2018-07-25
...,...,...,...,...
178,王的血族,97,20,2007-10-03
257,真偽贋物遊覧会,97,20,2012-07-18
281,PANDEMIC,97,20,2014-08-27


In [445]:
highest_top = target.sort_values(['rank_highest_exclude_debut', 'date'], ascending=[True, True]) \
                    .query('highest_exclude_debut <= 20') \
                    [['title', 'rank_highest_exclude_debut', 'highest_exclude_debut', 'date']].astype({'rank_highest_exclude_debut': int, 'highest_exclude_debut': int})

highest_top

,title,rank_highest_exclude_debut,highest_exclude_debut,date
no,,,,
132,北京LOVERS,1,1,2005-06-22
135,聖少女領域,1,1,2005-10-26
107,月蝕グランギニョル,3,2,2003-11-05
179,逢魔ヶ恋,3,2,2007-10-03
145,GOD DIVA,5,3,2006-06-28
152,勇侠青春謳,5,3,2006-10-25
183,わが﨟たし悪の華,5,3,2008-07-30
282,Lolicate,5,3,2014-08-27
108,未來のイヴ,9,4,2003-11-05


In [446]:
tmp = target.loc[:]
tmp['under10'] = 0
for no in tmp.index:
    song = tmp.loc[no]
    to_mean_all = list(filter(lambda y: year_filter(y, song.initial, RANK_ALL), RANK_ALL))
    tmp.under10.at[no] = len(list(filter(lambda s: s <= 10, song[to_mean_all].values)))
tmp['under10_mean'] = tmp.under10 / tmp.count_all
tmp.sort_values(['under10_mean', 'date'], ascending=[False, True]) \
   .query('under10_mean >= 1') \
   [['title', 'under10', 'under10_mean', 'date']]

,title,under10,under10_mean,date
no,,,,
107,月蝕グランギニョル,16,1.0,2003-11-05
135,聖少女領域,15,1.0,2005-10-26
191,雪華懺悔心中,11,1.0,2008-08-27
326,芸術変態論,2,1.0,2018-07-25
338,BArADiPArADicA,1,1.0,2019-08-28
343,自由戀愛,1,1.0,2019-08-28


In [447]:
tmp = target.loc[:]
tmp['under20'] = 0
for no in tmp.index:
    song = tmp.loc[no]
    to_mean_all = list(filter(lambda y: year_filter(y, song.initial, RANK_ALL), RANK_ALL))
    tmp.under20.at[no] = len(list(filter(lambda s: s <= 20, song[to_mean_all].values)))
tmp['under20_mean'] = tmp.under20 / tmp.count_all
tmp.sort_values(['under20_mean', 'date'], ascending=[False, True]) \
   .query('under20_mean >= 1') \
   [['title', 'under20', 'under20_mean', 'date']]

,title,under20,under20_mean,date
no,,,,
107,月蝕グランギニョル,16,1.0,2003-11-05
108,未來のイヴ,16,1.0,2003-11-05
135,聖少女領域,15,1.0,2005-10-26
191,雪華懺悔心中,11,1.0,2008-08-27
282,Lolicate,6,1.0,2014-08-27
322,卑弥呼外伝,3,1.0,2017-03-29
326,芸術変態論,2,1.0,2018-07-25
338,BArADiPArADicA,1,1.0,2019-08-28
339,21世紀新青年,1,1.0,2019-08-28


In [448]:
tmp = target.loc[:]
tmp['under30_mean'] = tmp.under30 / tmp.count_all
tmp.sort_values(['under30_mean', 'date'], ascending=[False, True]) \
   .query('under30_mean >= 1') \
   [['title', 'under30', 'under30_mean', 'date']]

,title,under30,under30_mean,date
no,,,,
107,月蝕グランギニョル,16,1.0,2003-11-05
108,未來のイヴ,16,1.0,2003-11-05
135,聖少女領域,15,1.0,2005-10-26
191,雪華懺悔心中,11,1.0,2008-08-27
251,凶夢伝染,8,1.0,2012-01-25
282,Lolicate,6,1.0,2014-08-27
319,まだら恋椿外道,4,1.0,2016-08-24
322,卑弥呼外伝,3,1.0,2017-03-29
326,芸術変態論,2,1.0,2018-07-25


In [449]:
tmp = target.loc[:, :]
tmp['under50_mean'] = tmp.under50 / tmp.count_all
tmp.sort_values(['under50_mean', 'date'], ascending=[False, True]) \
   .query('under50_mean >= 1') \
   [['title', 'under50', 'under50_mean', 'date']]

,title,under50,under50_mean,date
no,,,,
99,戦争と平和,16,1.0,2002-07-24
107,月蝕グランギニョル,16,1.0,2003-11-05
108,未來のイヴ,16,1.0,2003-11-05
112,地獄の季節,16,1.0,2003-12-03
121,禁じられた遊び,16,1.0,2004-10-22
135,聖少女領域,15,1.0,2005-10-26
145,GOD DIVA,14,1.0,2006-06-28
149,KING KNIGHT,14,1.0,2006-09-21
152,勇侠青春謳,14,1.0,2006-10-25


In [450]:
tmp = target.loc[:, :]
tmp['under100_mean'] = tmp.under100 / tmp.count_all
tmp.sort_values(['under100_mean', 'date'], ascending=[False, True]) \
   .query('under100_mean >= 1') \
   [['title', 'under100', 'under100_mean', 'date']]

,title,under100,under100_mean,date
no,,,,
24,嵐ヶ丘,16,1.0,1993-06-09
72,コッペリアの柩,16,1.0,1999-12-24
99,戦争と平和,16,1.0,2002-07-24
107,月蝕グランギニョル,16,1.0,2003-11-05
108,未來のイヴ,16,1.0,2003-11-05
112,地獄の季節,16,1.0,2003-12-03
121,禁じられた遊び,16,1.0,2004-10-22
126,人生美味礼讃,16,1.0,2005-06-22
135,聖少女領域,15,1.0,2005-10-26


#### first "over30"

In [451]:
target[target[RECENT] > 30].query('over30 == 1')

,title,ruby,inst,hall_of_fame,works,debut,date,r2019u,initial,kind,...,under100,under200,count_all,count,rank_average,rank_average_exclude_debut,rank_highest,rank_highest_exclude_debut,under50_mean,under100_mean
no,,,,,,,,,,,,,,,,,,,,,
328,ヤマトイズム,None,False,False,[芸術変態論],芸術変態論,2018-07-25,NaN,r2018,[OA],...,1,2,2,2,124.0,204.0,116.0,280.0,0.5,0.5
332,極まれリ,None,False,False,[芸術変態論],芸術変態論,2018-07-25,NaN,r2018,[OA],...,2,2,2,2,50.0,85.0,50.0,183.0,0.5,1.0
333,或る修道士の告解,None,False,False,[芸術変態論],芸術変態論,2018-07-25,NaN,r2018,[OA],...,2,2,2,2,29.0,26.0,97.0,72.0,1.0,1.0
337,美しき獣たちの為の,None,False,False,[『コードギアス 復活のルルーシュ』オリジナルサウンドトラック],『コードギアス 復活のルルーシュ』オリジナルサウンドトラック,2019-02-13,NaN,r2019,[ST],...,1,1,1,1,81.0,301.0,193.0,301.0,0.0,1.0
342,月下、緑雨幻想,None,False,False,[Fantasia],Fantasia,2019-08-28,NaN,r2019,[OA],...,0,1,1,1,183.0,301.0,281.0,301.0,0.0,0.0
345,恐怖の頭脳改革,None,False,False,[Fantasia],Fantasia,2019-08-28,NaN,r2019,[OA],...,1,1,1,1,69.0,301.0,178.0,301.0,0.0,1.0
346,地球で,テラで,False,False,[Fantasia],Fantasia,2019-08-28,NaN,r2019,[OA],...,1,1,1,1,126.0,301.0,233.0,301.0,0.0,1.0
348,WALTZ FOR DEBBY (Bill Evans),None,False,False,[Fantasia],Fantasia,2019-08-28,NaN,r2019,[OA],...,1,1,1,1,55.0,301.0,171.0,301.0,0.0,1.0


#### first "over50"

In [452]:
target[target[RECENT] > 50].query('over50 == 1')

,title,ruby,inst,hall_of_fame,works,debut,date,r2019u,initial,kind,...,under100,under200,count_all,count,rank_average,rank_average_exclude_debut,rank_highest,rank_highest_exclude_debut,under50_mean,under100_mean
no,,,,,,,,,,,,,,,,,,,,,
185,神風,None,False,False,"[禁書, 愛と誠 ～YAMATO＆LOVE×××]",禁書,2008-08-27,NaN,r2009,"[OA, CA]",...,10,11,11,11,37.0,31.0,61.0,30.0,0.909091,0.909091
327,Taolor Tの変身譚,None,False,False,[芸術変態論],芸術変態論,2018-07-25,NaN,r2018,[OA],...,1,2,2,2,113.0,153.0,152.0,249.0,0.500000,0.500000
328,ヤマトイズム,None,False,False,[芸術変態論],芸術変態論,2018-07-25,NaN,r2018,[OA],...,1,2,2,2,124.0,204.0,116.0,280.0,0.500000,0.500000
329,少女昆蟲記,None,False,False,[芸術変態論],芸術変態論,2018-07-25,NaN,r2018,[OA],...,2,2,2,2,44.0,42.0,138.0,113.0,0.500000,1.000000
332,極まれリ,None,False,False,[芸術変態論],芸術変態論,2018-07-25,NaN,r2018,[OA],...,2,2,2,2,50.0,85.0,50.0,183.0,0.500000,1.000000
334,タンゴ ダタ,None,False,False,[芸術変態論],芸術変態論,2018-07-25,NaN,r2018,[OA],...,1,2,2,2,102.0,146.0,132.0,243.0,0.500000,0.500000
337,美しき獣たちの為の,None,False,False,[『コードギアス 復活のルルーシュ』オリジナルサウンドトラック],『コードギアス 復活のルルーシュ』オリジナルサウンドトラック,2019-02-13,NaN,r2019,[ST],...,1,1,1,1,81.0,301.0,193.0,301.0,0.000000,1.000000
342,月下、緑雨幻想,None,False,False,[Fantasia],Fantasia,2019-08-28,NaN,r2019,[OA],...,0,1,1,1,183.0,301.0,281.0,301.0,0.000000,0.000000
345,恐怖の頭脳改革,None,False,False,[Fantasia],Fantasia,2019-08-28,NaN,r2019,[OA],...,1,1,1,1,69.0,301.0,178.0,301.0,0.000000,1.000000


#### first "over100"

In [453]:
target[target[RECENT] > 100].query('over100 == 1')

,title,ruby,inst,hall_of_fame,works,debut,date,r2019u,initial,kind,...,under100,under200,count_all,count,rank_average,rank_average_exclude_debut,rank_highest,rank_highest_exclude_debut,under50_mean,under100_mean
no,,,,,,,,,,,,,,,,,,,,,
130,昭和恋々幻燈館,None,False,False,[Dilettante],Dilettante,2005-06-22,NaN,r2005,[OA],...,12,13,16,13,72.0,53.0,136.0,87.0,0.187500,0.750000
181,コトダマ,None,False,False,"[コトダマ, シゴフミ オリジナルサウンドトラック, La Vita Romantica, ...",コトダマ,2008-01-23,NaN,r2008,"[S-1, ST, CA]",...,11,11,12,12,71.0,60.0,39.0,72.0,0.416667,0.916667
185,神風,None,False,False,"[禁書, 愛と誠 ～YAMATO＆LOVE×××]",禁書,2008-08-27,NaN,r2009,"[OA, CA]",...,10,11,11,11,37.0,31.0,61.0,30.0,0.909091,0.909091
261,真夏の憂愁夫人,None,False,False,"[贋作師, 愛と誠 ～YAMATO＆LOVE×××]",贋作師,2012-07-18,NaN,r2012,"[OA, CA]",...,7,8,8,8,42.0,37.0,44.0,18.0,0.750000,0.875000
264,快恠奇奇,None,False,False,"[快恠奇奇 ALI PROJECT Ventennale Music, Art Exhibi...","快恠奇奇 ALI PROJECT Ventennale Music, Art Exhibition",2013-02-20,NaN,r2013,[CA],...,6,7,7,7,63.0,58.0,61.0,76.0,0.428571,0.857143
268,令嬢薔薇図鑑,None,False,False,"[令嬢薔薇図鑑, 血と蜜 ～Anthology of Gothic Lolita＆Horror]",令嬢薔薇図鑑,2013-09-11,NaN,r2013,"[OA, CA]",...,6,7,7,7,54.0,50.0,30.0,56.0,0.571429,0.857143
269,ローズ家の双子達,None,False,False,"[令嬢薔薇図鑑, 血と蜜 ～Anthology of Gothic Lolita＆Horror]",令嬢薔薇図鑑,2013-09-11,NaN,r2013,"[OA, CA]",...,6,7,7,7,45.0,45.0,18.0,26.0,0.714286,0.857143
317,昭和B級下手喰い道,None,False,False,[A級戒厳令],A級戒厳令,2016-08-24,NaN,r2016,[OA],...,3,4,4,4,66.0,74.0,30.0,94.0,0.500000,0.750000
318,女化生舞楽図,おんなけしょうぶがくず,False,False,"[A級戒厳令, 血と蜜 ～Anthology of Gothic Lolita＆Horror]",A級戒厳令,2016-08-24,NaN,r2016,"[OA, CA]",...,3,4,4,4,80.0,82.0,82.0,101.0,0.500000,0.750000


#### first "over200"

In [454]:
target[target[RECENT] > 200].query('over200 == 1')

,title,ruby,inst,hall_of_fame,works,debut,date,r2019u,initial,kind,...,under100,under200,count_all,count,rank_average,rank_average_exclude_debut,rank_highest,rank_highest_exclude_debut,under50_mean,under100_mean
no,,,,,,,,,,,,,,,,,,,,,
8,パラソルのある風景,None,False,False,[幻想庭園],幻想庭園,1988-01-25,109.0,r2005,[OA],...,0,12,16,13,236.0,219.0,268.0,241.0,0.000000,0.000000
38,冬物語,None,False,False,[星と月のソナタ],星と月のソナタ,1995-12-06,86.0,r2005,[OA],...,1,12,16,13,187.0,169.0,216.0,179.0,0.000000,0.062500
40,ストロベリーパイをお食べ,None,False,False,"[緑山高校甲子園編 オリジナルサウンドトラック盤, jamais vu]",緑山高校甲子園編 オリジナルサウンドトラック盤,1995-12-06,63.0,r2005,"[ST, CA]",...,6,12,16,13,143.0,123.0,183.0,137.0,0.000000,0.375000
96,schism,None,False,False,[EROTIC & HERETIC],EROTIC & HERETIC,2002-07-24,NaN,r2005,[OA],...,8,12,16,13,130.0,105.0,151.0,100.0,0.062500,0.500000
181,コトダマ,None,False,False,"[コトダマ, シゴフミ オリジナルサウンドトラック, La Vita Romantica, ...",コトダマ,2008-01-23,NaN,r2008,"[S-1, ST, CA]",...,11,11,12,12,71.0,60.0,39.0,72.0,0.416667,0.916667
189,血の断章,None,False,False,"[禁書, 血と蜜 ～Anthology of Gothic Lolita＆Horror]",禁書,2008-08-27,73.0,r2009,"[OA, CA]",...,2,10,11,11,205.0,189.0,226.0,194.0,0.000000,0.181818
202,戦慄の子供たち,None,False,False,"[戦慄の子供たち, Phantom ～Requiem for the Phantom～ O....",戦慄の子供たち,2009-08-19,59.0,r2010,"[S-1, ST, CA]",...,6,9,10,10,120.0,110.0,113.0,122.0,0.100000,0.600000
209,Animals on the Earth,None,False,False,[Poison],Poison,2009-08-26,50.0,r2010,[OA],...,5,9,10,10,146.0,136.0,132.0,97.0,0.200000,0.500000
252,夜見のたそがれの、うつろなる蒼き瞳の。,None,False,False,"[凶夢伝染, 血と蜜 ～Anthology of Gothic Lolita＆Horror]",凶夢伝染,2012-01-25,51.0,r2012,"[S-2, CA]",...,4,7,8,8,159.0,142.0,200.0,157.0,0.000000,0.500000


### no voted
#### last year

In [455]:
lowest[BEFORE] and target[target[BEFORE] == lowest[BEFORE]]

,title,ruby,inst,hall_of_fame,works,debut,date,r2019u,initial,kind,...,under100,under200,count_all,count,rank_average,rank_average_exclude_debut,rank_highest,rank_highest_exclude_debut,under50_mean,under100_mean
no,,,,,,,,,,,,,,,,,,,,,
4,鏡面界 im Juni,きょうめんかい・イン・ジュニ,False,False,[幻想庭園],幻想庭園,1988-01-25,150.0,r2005,[OA],...,0,7,16,13,276.0,252.0,269.0,242.0,0.000000,0.000000
20,木洩れ陽のワルツ,None,False,False,[月下の一群],月下の一群,1992-12-09,131.0,r2005,[OA],...,0,5,16,13,290.0,272.0,301.0,278.0,0.000000,0.000000
22,月光浴,None,False,False,"[月下の一群, 星と月のソナタ, Déjà Vu ～THE ORIGINAL BEST 19...",月下の一群,1992-12-09,179.0,r2005,"[OA, CA]",...,0,10,16,13,243.0,224.0,257.0,229.0,0.000000,0.000000
23,楽園喪失 -Paradise Lost-,None,False,False,"[月下の一群, Romance]",月下の一群,1992-12-09,128.0,r2005,"[SA, OA]",...,0,7,16,13,260.0,240.0,252.0,220.0,0.000000,0.000000
32,小夜啼鳥,ナイチンゲール,False,False,[DALI],DALI,1994-02-16,174.0,r2005,[OA],...,0,4,16,13,282.0,264.0,265.0,238.0,0.000000,0.000000
45,エンジェル・エッグの作り方,None,False,False,[music tracks from Wish],music tracks from Wish,1997-02-05,132.0,r2005,[ST],...,0,9,16,13,266.0,246.0,284.0,256.0,0.000000,0.000000
46,Chu Chu,None,False,False,[music tracks from Wish],music tracks from Wish,1997-02-05,150.0,r2005,[ST],...,0,2,16,13,304.0,289.0,278.0,249.0,0.000000,0.000000
47,Sanctuary,None,False,False,[music tracks from Wish],music tracks from Wish,1997-02-05,181.0,r2005,[ST],...,0,2,16,13,309.0,292.0,310.0,293.0,0.000000,0.000000
50,Water Drop,None,False,False,"[CLAMP学園探偵団 O.S.T 1, CLAMP学園探偵団 Vocal Collecti...",CLAMP学園探偵団 O.S.T 1,1997-06-21,194.0,r2005,"[ST, OT]",...,0,5,16,13,296.0,276.0,288.0,266.0,0.000000,0.000000


#### this year

In [456]:
lowest[RECENT] and target[target[RECENT] == lowest[RECENT]]

,title,ruby,inst,hall_of_fame,works,debut,date,r2019u,initial,kind,...,under100,under200,count_all,count,rank_average,rank_average_exclude_debut,rank_highest,rank_highest_exclude_debut,under50_mean,under100_mean
no,,,,,,,,,,,,,,,,,,,,,
4,鏡面界 im Juni,きょうめんかい・イン・ジュニ,False,False,[幻想庭園],幻想庭園,1988-01-25,150.0,r2005,[OA],...,0,7,16,13,276.0,252.0,269.0,242.0,0.000000,0.000000
8,パラソルのある風景,None,False,False,[幻想庭園],幻想庭園,1988-01-25,109.0,r2005,[OA],...,0,12,16,13,236.0,219.0,268.0,241.0,0.000000,0.000000
12,フラワーチャイルド,None,False,False,"[フラワーチャイルド, 幻想庭園, 快恠奇奇 ALI PROJECT Ventennale ...",フラワーチャイルド,1988-07-25,165.0,r2005,"[OA, S-1, CA]",...,0,5,16,13,283.0,265.0,272.0,243.0,0.000000,0.000000
20,木洩れ陽のワルツ,None,False,False,[月下の一群],月下の一群,1992-12-09,131.0,r2005,[OA],...,0,5,16,13,290.0,272.0,301.0,278.0,0.000000,0.000000
23,楽園喪失 -Paradise Lost-,None,False,False,"[月下の一群, Romance]",月下の一群,1992-12-09,128.0,r2005,"[SA, OA]",...,0,7,16,13,260.0,240.0,252.0,220.0,0.000000,0.000000
31,Virtual Fantasy,None,False,False,[DALI],DALI,1994-02-16,148.0,r2005,[OA],...,0,8,16,13,257.0,237.0,259.0,231.0,0.000000,0.000000
44,夢のあとに ～après un rêve～,None,False,False,"[Wish, music tracks from Wish, CLAMPAZAR ～クランパ...",Wish,1996-12-24,162.0,r2005,"[S-2, ST, OT, CA]",...,0,6,16,13,284.0,268.0,300.0,276.0,0.000000,0.000000
45,エンジェル・エッグの作り方,None,False,False,[music tracks from Wish],music tracks from Wish,1997-02-05,132.0,r2005,[ST],...,0,9,16,13,266.0,246.0,284.0,256.0,0.000000,0.000000
46,Chu Chu,None,False,False,[music tracks from Wish],music tracks from Wish,1997-02-05,150.0,r2005,[ST],...,0,2,16,13,304.0,289.0,278.0,249.0,0.000000,0.000000


### highest ever
(exclude recent year)

In [457]:
target[target.highest == target[RECENT]].query('initial != @RECENT')[['title', *RANK_ALL]]

,title,r2005,r2006#1,r2006#2,r2007,r2008,r2009,r2010,r2011,r2012,r2013,r2014,r2015,r2016,r2017,r2018,r2019
no,,,,,,,,,,,,,,,,,
2,青蛾月,NaN,NaN,NaN,84.0,77.0,73.0,82.0,72.0,57.0,54.0,87.0,81.0,59.0,146.0,126.0,54.0
102,Nostalgia,NaN,NaN,NaN,151.0,145.0,114.0,135.0,128.0,134.0,110.0,142.0,172.0,131.0,199.0,133.0,77.0
157,今宵、碧い森深く,NaN,NaN,NaN,23.0,23.0,27.0,30.0,20.0,38.0,32.0,32.0,30.0,27.0,29.0,16.0,13.0
292,修道院の廃庭にて,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,167.0,225.0,161.0,214.0,101.0


### lowest ever
(voted in recent year)

In [458]:
target[(target.lowest == target[RECENT]) & (target[RECENT] != lowest[RECENT])].query('initial != @RECENT')[['title', *RANK_ALL]]

,title,r2005,r2006#1,r2006#2,r2007,r2008,r2009,r2010,r2011,r2012,r2013,r2014,r2015,r2016,r2017,r2018,r2019
no,,,,,,,,,,,,,,,,,
27,エスカルゴ嬉遊曲,NaN,NaN,NaN,56.0,66.0,91.0,109.0,155.0,194.0,161.0,188.0,215.0,216.0,213.0,200.0,226.0
38,冬物語,NaN,NaN,NaN,135.0,128.0,135.0,155.0,158.0,150.0,155.0,130.0,200.0,84.0,108.0,103.0,226.0
40,ストロベリーパイをお食べ,NaN,NaN,NaN,68.0,63.0,74.0,81.0,89.0,127.0,82.0,150.0,115.0,136.0,118.0,151.0,226.0
96,schism,NaN,NaN,NaN,46.0,74.0,71.0,104.0,97.0,104.0,93.0,76.0,189.0,115.0,59.0,57.0,226.0
98,時の森のソワレ,17.0,NaN,NaN,66.0,88.0,86.0,85.0,84.0,94.0,123.0,109.0,106.0,138.0,146.0,136.0,159.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,青空,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,24.0
332,極まれリ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,86.0
333,或る修道士の告解,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,32.0


### diff

In [459]:
diff = target.query('initial != @RECENT')
diff['diff'] = diff[BEFORE] - diff[RECENT]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### up

In [460]:
diff.sort_values('diff', ascending=False)[['title', 'diff']].head(10)

,title,diff
no,,
77,少女貴族,134.0
262,天譴と超克,133.0
15,メガロポリス・アリス -MEGALOPOLIS ALICE-,131.0
270,隼の白バラ,125.0
133,鎮魂頌,125.0
292,修道院の廃庭にて,113.0
33,星降る夜の天文学 ～BEDSIDE ASTRONOMY～,107.0
62,Halation,91.0
254,贋作師,90.0


### down

In [461]:
diff.sort_values('diff')[['title', 'diff']].head(10)

,title,diff
no,,
96,schism,-169.0
328,ヤマトイズム,-144.0
272,朗読する女中と小さな令嬢,-138.0
181,コトダマ,-128.0
38,冬物語,-123.0
201,吾君想う故に吾在り生き霊となりて,-122.0
170,CYBER DEVILS,-121.0
104,プラトニック,-112.0
325,花と龍,-105.0


### top average works
#### S

In [462]:
dfw.query('kind == "S"').sort_values(f'{RECENT}_average')

,title,kind,date,r2005_highest,r2006#1_highest,r2006#2_highest,r2007_average,r2007_highest,r2008_average,r2008_highest,...,r2015_average,r2015_highest,r2016_average,r2016_highest,r2017_average,r2017_highest,r2018_average,r2018_highest,r2019_average,r2019_highest
33,月蝕グランギニョル,S,2003-11-05,5.0,3.0,5.0,9.0,4.0,12.0,5.0,...,8.000000,6.0,6.000000,4.0,9.000000,2.0,3.500000,3.0,6.500000,5.0
111,卑弥呼外伝,S,2017-03-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4.000000,4.0,12.000000,12.0,18.000000,18.0
29,コッペリアの柩 (Hyper Remix Version),S,2001-09-21,7.0,17.0,17.0,22.0,22.0,43.0,43.0,...,38.000000,38.0,58.000000,58.0,85.000000,85.0,78.000000,78.0,42.000000,42.0
56,勇侠青春謳,S,2006-10-25,NaN,NaN,1.0,16.5,3.0,36.0,4.0,...,71.500000,24.0,47.000000,24.0,41.500000,18.0,130.000000,39.0,54.000000,12.0
62,跪いて足をお嘗め,S,2007-06-13,NaN,NaN,NaN,NaN,NaN,33.0,9.0,...,115.500000,56.0,137.000000,30.0,143.000000,49.0,115.000000,30.0,81.000000,38.0
3,恋せよ乙女 -Love story of ZIPANG-,S,1992-07-07,13.0,13.0,NaN,37.0,36.0,48.5,48.0,...,62.000000,31.0,138.500000,52.0,66.500000,20.0,41.000000,24.0,88.000000,44.0
69,わが臈たし悪の華,S,2008-07-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,110.500000,26.0,95.500000,31.0,102.500000,29.0,117.500000,21.0,89.500000,23.0
26,コッペリアの柩,S,2001-05-23,7.0,17.0,17.0,50.0,22.0,70.0,43.0,...,85.000000,38.0,100.000000,58.0,113.000000,85.0,116.000000,78.0,99.500000,42.0
58,薔薇獄乙女,S,2006-12-06,NaN,NaN,2.0,18.0,8.0,51.0,17.0,...,88.000000,37.0,133.000000,41.0,123.500000,34.0,102.500000,61.0,101.000000,35.0
11,星月夜 ～ルシファー第四楽章～,S,1996-08-01,NaN,NaN,NaN,138.0,89.0,116.5,104.0,...,122.500000,45.0,134.000000,67.0,123.000000,44.0,129.500000,59.0,103.000000,36.0


#### OA, SA, CA

In [463]:
dfw.query('kind in ["OA", "SA", "CA"]').sort_values(f'{RECENT}_average')[['title', f'{BEFORE}_average', f'{RECENT}_average']]

,title,r2018_average,r2019_average
116,Fantasia,NaN,41.900000
43,Dilettante,93.875000,77.000000
114,芸術変態論,30.100000,83.300000
73,桂冠詩人 SINGLE COLLECTION PLUS,110.230769,94.846154
98,"快恠奇奇 ALI PROJECT Ventennale Music, Art Exhibition",74.709677,96.677419
66,Grand Finale,107.571429,99.428571
63,Psychedelic Insanity,97.444444,102.888889
31,EROTIC & HERETIC,109.500000,109.200000
109,A級戒厳令,89.777778,113.777778
113,愛と誠 ～YAMATO＆LOVE×××,115.580645,115.741935


#### I voted

In [464]:
my = ['ナルシス・ノワール', '月蝕グランギニョル', '少女殉血', '未來のイヴ', '地獄の季節', '熱帯性植物園', '遊月恋歌', '戦争と平和', '暗黒天国', '胡蝶夢心中', '六道輪廻サバイバル', '魅惑劇']
target.query('title in @my')

,title,ruby,inst,hall_of_fame,works,debut,date,r2019u,initial,kind,...,under100,under200,count_all,count,rank_average,rank_average_exclude_debut,rank_highest,rank_highest_exclude_debut,under50_mean,under100_mean
no,,,,,,,,,,,,,,,,,,,,,
66,ナルシス・ノワール,None,False,False,"[Noblerot, 月光嗜好症]",Noblerot,1998-11-21,NaN,r2005,"[SA, OA]",...,13,14,16,13,47.0,38.0,57.0,46.0,0.625000,0.812500
94,熱帯性植物園,None,False,False,[EROTIC & HERETIC],EROTIC & HERETIC,2002-07-24,26.0,r2005,[OA],...,6,13,16,13,141.0,122.0,161.0,113.0,0.000000,0.375000
95,遊月恋歌,None,False,False,"[EROTIC & HERETIC, Romance, 愛と誠 ～YAMATO＆LOVE×××]",EROTIC & HERETIC,2002-07-24,2.0,r2005,"[SA, OA, CA]",...,15,16,16,13,48.0,39.0,61.0,38.0,0.750000,0.937500
99,戦争と平和,None,False,False,"[EROTIC & HERETIC, Grand Finale, 愛と誠 ～YAMATO＆L...",EROTIC & HERETIC,2002-07-24,NaN,r2005,"[SA, OA, CA]",...,16,16,16,13,19.0,13.0,1.0,14.0,1.000000,1.000000
107,月蝕グランギニョル,None,False,False,"[月蝕グランギニョル, Avenger O.S.T., COLLECTION SIMPLE ...",月蝕グランギニョル,2003-11-05,NaN,r2005,"[S-1, SA, ST, CA]",...,16,16,16,13,5.0,2.0,7.0,3.0,1.000000,1.000000
108,未來のイヴ,None,False,False,"[月蝕グランギニョル, Avenger O.S.T., COLLECTION SIMPLE ...",月蝕グランギニョル,2003-11-05,NaN,r2005,"[S-2, ST, CA]",...,16,16,16,13,10.0,6.0,23.0,9.0,1.000000,1.000000
111,少女殉血,None,False,False,"[Avenger O.S.T., 桂冠詩人 SINGLE COLLECTION PLUS, ...",Avenger O.S.T.,2003-12-03,4.0,r2005,"[ST, CA]",...,8,14,16,13,140.0,121.0,53.0,62.0,0.250000,0.500000
112,地獄の季節,None,False,False,"[Avenger O.S.T., COLLECTION SIMPLE PLUS]",Avenger O.S.T.,2003-12-03,NaN,r2005,"[ST, CA]",...,16,16,16,13,27.0,21.0,30.0,28.0,1.000000,1.000000
164,暗黒天国,None,False,False,"[暗黒天国, かみちゃまかりん オリジナル・サウンドトラック, La Vita Romant...",暗黒天国,2007-05-23,NaN,r2008,"[S-1, ST, CA]",...,11,12,12,12,61.0,52.0,90.0,44.0,0.416667,0.916667


# URA ranking

In [465]:
# ura ranking
ura = df[df[URA] > 100][COLUMNS]
ura['rank_average'] = ura.average.rank(method='min', na_option='bottom')
ura['rank_average_exclude_debut'] = ura.average_exclude_debut.rank(method='min', na_option='bottom')
ura['rank_highest'] = ura.highest.rank(method='min', na_option='bottom')
ura['rank_highest_exclude_debut'] = ura.highest_exclude_debut.rank(method='min', na_option='bottom')

ura.head()

,title,date,initial,kind,r2007,r2008,r2009,r2010,r2011,r2012,...,under30,under50,under100,under200,count_all,count,rank_average,rank_average_exclude_debut,rank_highest,rank_highest_exclude_debut
no,,,,,,,,,,,,,,,,,,,,,
1,桜の花は狂い咲き,1997-04-25,r2005,"[SA, OA, OT]",86.0,126.0,116.0,109.0,96.0,123.0,...,0,0,4,13,16,13,58.0,46.0,97.0,64.0
3,マリーゴールド・ガーデン,1988-01-25,r2005,"[SA, OA, S-2]",141.0,162.0,142.0,150.0,175.0,141.0,...,0,0,0,10,16,13,126.0,120.0,183.0,164.0
4,鏡面界 im Juni,1988-01-25,r2005,[OA],165.0,185.0,181.0,171.0,204.0,131.0,...,0,0,0,7,16,13,176.0,162.0,169.0,152.0
5,アンジェ・ノワールの祭戯,1988-01-25,r2005,"[SA, OA]",109.0,131.0,111.0,93.0,81.0,96.0,...,0,0,4,12,16,13,65.0,52.0,87.0,55.0
6,紅い睡蓮の午後,1988-01-25,r2005,"[SA, OA]",144.0,176.0,153.0,164.0,188.0,204.0,...,0,0,0,8,16,13,150.0,143.0,178.0,159.0


In [466]:
col = 4
length = len(ura)
row = int(pandas.np.ceil(length/col))

a = [[] for _ in range(col)]
c = 0
for no in ura.index:
    a[int(c // row)] += [ura.title.at[no]]
    c += 1

for i in range(row):
    print('|'.join([a[j][i] if j * row + (i + 1) <= length else '' for j in range(col)]))

桜の花は狂い咲き|病める薔薇|眠れる豹|真偽贋物遊覧会
マリーゴールド・ガーデン|MALICE|血の断章|La vérité
鏡面界 im Juni|à la cuisine|薨|野性双生児
アンジェ・ノワールの祭戯|桂花葬|小さき者への贖罪のためのアリア|逝ける王女の肖像画
紅い睡蓮の午後|閉ざされた画室|鬼帝の剣|真夏の憂愁夫人
少女忌恋歌|un tableau blanc ～絵画旅行～|桂冠詩人|天譴と超克
パラソルのある風景|闇の翼ですべてをつつむ夜のためのアリア|裸々イヴ新世紀|快恠奇奇
硝子天井のうちゅう|après le noir|騎士乙女|Fräulein Rose
幻想庭園|宵待草|吾君想う故に吾在り生き霊となりて|乙女の贖い
フラワーチャイルド|La vie en rose|戦慄の子供たち|令嬢薔薇図鑑
恋せよ乙女 -Love story of ZIPANG-|schism|Hell's Maria|ローズ家の双子達
ビアンカ|時の森のソワレ|処女懐胎、あるいは白骨塔より少女達は飛翔する|隼の白バラ
空宙舞踏会|夢魔の夜、あなたを迎える者がある|お毒味LADY|薔薇娼館
木洩れ陽のワルツ|プラトニック|Animals on the Earth|朗読する女中と小さな令嬢
堕ちて候|Féliciter|上海繚乱ロマンチカ|黒百合隠密カゲキダン
月光浴|MOTHER|世紀末ゲネシス|南無地獄大菩薩
楽園喪失 -Paradise Lost-|繭|この國の向こうに|見ぬ友へ
舞踏会の手帖|Je te veux|輪廻闇妖散華|PANDEMIC
エスカルゴ嬉遊曲|Après un rêve|堕天國宣戦|赤い蝋燭と金魚
サロメティック・ルナティック|Daphné|寶島|Femme Fatale
雪のひとひら|わが恋は月光に散りぬ|月光オペラの天鵞絨の緞帳は上がり|海賊皇女
ダリの宝石店|pastel pure|恋の翼は真珠色の骨を持ち|静謐なる私小説
Virtual Fantasy|あたしがアリスだった頃|私の心は語る薔薇の御胸を葩の御身を|形而上的な、蝶になる
小夜啼鳥|君がため、惜しからざりし命さへ|エルフの娘は地上に降り|極楽鳥歌
星降る夜の天文学 ～BEDSIDE ASTRONOMY～|肉体の悪魔|百合の日々は追憶の中に潜み薫る|遊蝶花ワルツ
オフェリア

In [467]:
average_top = ura.sort_values(['rank_average_exclude_debut', 'date'], ascending=[True, True]) \
                 .query('average_exclude_debut <= 100') \
                 [['title', 'rank_average_exclude_debut', 'average_exclude_debut', 'date']].astype({'rank_average_exclude_debut': int})

average_top

,title,rank_average_exclude_debut,average_exclude_debut,date
no,,,,
185,神風,1,39.400000,2008-08-27
261,真夏の憂愁夫人,2,44.857143,2012-07-18
269,ローズ家の双子達,3,52.333333,2013-09-11
268,令嬢薔薇図鑑,4,59.166667,2013-09-11
130,昭和恋々幻燈館,5,62.692308,2005-06-22
205,処女懐胎、あるいは白骨塔より少女達は飛翔する,6,63.222222,2009-08-26
195,鬼帝の剣,7,64.300000,2008-11-19
264,快恠奇奇,8,66.500000,2013-02-20
181,コトダマ,9,67.000000,2008-01-23


In [468]:
for no in average_top.index:
    print(f'{average_top.rank_average_exclude_debut.at[no]}|{average_top.title.at[no]}|{average_top.average_exclude_debut.at[no]:.2f}')

1|神風|39.40
2|真夏の憂愁夫人|44.86
3|ローズ家の双子達|52.33
4|令嬢薔薇図鑑|59.17
5|昭和恋々幻燈館|62.69
6|処女懐胎、あるいは白骨塔より少女達は飛翔する|63.22
7|鬼帝の剣|64.30
8|快恠奇奇|66.50
9|コトダマ|67.00
10|黒百合隠密カゲキダン|69.83
11|堕天國宣戦|72.67
12|昭和B級下手喰い道|78.33
13|お毒味LADY|79.89
14|女化生舞楽図|83.67
15|亂世エロイカ|86.12
16|ヘテロ失楽園|88.50
17|陸と海と空と|89.25
18|上海繚乱ロマンチカ|90.78
19|恋せよ乙女 -Love story of ZIPANG-|91.85
20|月光夜|96.46
21|平成日本残酷物語|96.62


In [469]:
highest_top = ura.sort_values(['rank_highest_exclude_debut', 'date'], ascending=[True, True]) \
                 .query('highest_exclude_debut <= 50') \
                 [['title', 'rank_highest_exclude_debut', 'highest_exclude_debut', 'highest', 'date']].astype({'rank_highest_exclude_debut': int, 'highest_exclude_debut': int, 'highest': int})

highest_top

,title,rank_highest_exclude_debut,highest_exclude_debut,highest,date
no,,,,,
261,真夏の憂愁夫人,1,9,9,2012-07-18
274,黒百合隠密カゲキダン,2,10,4,2013-09-11
269,ローズ家の双子達,3,12,3,2013-09-11
185,神風,4,14,13,2008-08-27
215,堕天國宣戦,4,14,7,2009-10-21
206,お毒味LADY,6,18,13,2009-08-26
205,処女懐胎、あるいは白骨塔より少女達は飛翔する,7,19,17,2009-08-26
195,鬼帝の剣,8,23,5,2008-11-19
268,令嬢薔薇図鑑,9,25,6,2013-09-11


In [470]:
tmp = ura.loc[:, :]
tmp['under100_mean'] = tmp.under100 / tmp.count_all
tmp.sort_values(['under100_mean', 'date'], ascending=[False, True]) \
   .query('under100_mean >= 0.8') \
   [['title', 'under100', 'under100_mean', 'date']]

,title,under100,under100_mean,date
no,,,,
181,コトダマ,11,0.916667,2008-01-23
185,神風,10,0.909091,2008-08-27
261,真夏の憂愁夫人,7,0.875000,2012-07-18
264,快恠奇奇,6,0.857143,2013-02-20
268,令嬢薔薇図鑑,6,0.857143,2013-09-11
269,ローズ家の双子達,6,0.857143,2013-09-11
195,鬼帝の剣,9,0.818182,2008-11-19
205,処女懐胎、あるいは白骨塔より少女達は飛翔する,8,0.800000,2009-08-26
215,堕天國宣戦,8,0.800000,2009-10-21


In [471]:
# 未投票
zero = ['日曜日のシエスタ', 'La vie en rose', 'Féliciter', 'Nocturnal', '寶島', '逢魔蛾城の伯爵', '小夜すみれ ～ある夜の森の小さなお話', '魂ノ代']
ura.query('title in @zero')

,title,date,initial,kind,r2007,r2008,r2009,r2010,r2011,r2012,...,under50,under100,under200,count_all,count,rank_average,rank_average_exclude_debut,rank_highest,rank_highest_exclude_debut,under100_mean
no,,,,,,,,,,,,,,,,,,,,,
55,日曜日のシエスタ,1997-12-17,r2005,"[SA, ST]",129.0,208.0,179.0,171.0,169.0,139.0,...,0,0,7,16,13,156.0,146.0,165.0,148.0,0.0
91,La vie en rose,2001-07-25,r2005,[OA],187.0,196.0,221.0,155.0,211.0,217.0,...,0,0,4,16,13,197.0,192.0,191.0,178.0,0.0
106,Féliciter,2003-04-23,r2005,[SA],168.0,174.0,215.0,179.0,223.0,185.0,...,0,0,5,16,13,180.0,170.0,201.0,188.0,0.0
180,Nocturnal,2007-12-12,r2008,[SA],NaN,154.0,202.0,192.0,223.0,217.0,...,0,0,2,12,12,205.0,204.0,188.0,204.0,0.0
216,寶島,2009-10-21,r2010,"[S-2, CA]",NaN,NaN,NaN,109.0,147.0,171.0,...,0,0,4,10,10,188.0,198.0,144.0,170.0,0.0
247,逢魔蛾城の伯爵,2011-06-29,r2011,[SA],NaN,NaN,NaN,NaN,123.0,158.0,...,0,0,4,9,9,161.0,161.0,162.0,165.0,0.0
296,小夜すみれ ～ある夜の森の小さなお話,2015-01-21,r2015,[SA],NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,2,5,5,192.0,191.0,187.0,174.0,0.0
311,魂ノ代,2015-10-21,r2015,[S-2],NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,1,5,5,211.0,210.0,204.0,210.0,0.0


In [472]:
ura[ura.kind.apply(lambda k: 'S-1' in k or 'S-2' in k)]

,title,date,initial,kind,r2007,r2008,r2009,r2010,r2011,r2012,...,under50,under100,under200,count_all,count,rank_average,rank_average_exclude_debut,rank_highest,rank_highest_exclude_debut,under100_mean
no,,,,,,,,,,,,,,,,,,,,,
3,マリーゴールド・ガーデン,1988-01-25,r2005,"[SA, OA, S-2]",141.0,162.0,142.0,150.0,175.0,141.0,...,0,0,10,16,13,126.0,120.0,183.0,164.0,0.000000
12,フラワーチャイルド,1988-07-25,r2005,"[OA, S-1, CA]",187.0,197.0,221.0,214.0,223.0,217.0,...,0,0,5,16,13,183.0,175.0,172.0,153.0,0.000000
13,恋せよ乙女 -Love story of ZIPANG-,1992-07-07,r2005,"[OA, S-1, SA, CA]",36.0,48.0,58.0,74.0,87.0,71.0,...,3,10,13,16,13,26.0,19.0,14.0,19.0,0.625000
25,舞踏会の手帖,1993-06-09,r2005,"[OA, S-2, CA]",92.0,137.0,174.0,171.0,223.0,153.0,...,0,1,10,16,13,141.0,132.0,125.0,101.0,0.062500
27,エスカルゴ嬉遊曲,1994-01-19,r2005,"[S-2, CA]",56.0,66.0,91.0,109.0,155.0,194.0,...,0,3,9,16,13,116.0,104.0,77.0,43.0,0.187500
37,Nous Deux C'est Pour La Vie,1995-10-18,r2005,"[S-2, CA]",44.0,65.0,80.0,118.0,117.0,107.0,...,2,5,14,16,13,60.0,48.0,20.0,26.0,0.312500
42,Only Love Song,1996-08-01,r2005,[S-2],187.0,129.0,185.0,150.0,158.0,217.0,...,0,0,10,16,13,139.0,130.0,165.0,148.0,0.000000
43,Wish,1996-12-24,r2005,"[S-1, OT, ST, CA]",53.0,71.0,81.0,72.0,86.0,127.0,...,1,6,13,16,13,52.0,42.0,67.0,33.0,0.375000
44,夢のあとに ～après un rêve～,1996-12-24,r2005,"[S-2, ST, OT, CA]",168.0,199.0,198.0,164.0,211.0,217.0,...,0,0,6,16,13,184.0,178.0,200.0,186.0,0.000000


In [473]:
# ura_list[(160 <= ura_list[2018]) & (ura_list[2018] <= 210)]

In [474]:
# len(_)

In [475]:
my_before = ['桜の花は狂い咲き', '乙女の祈り', 'Rose Moon', '少女貴族', '熱帯性植物園', '遊月恋歌', '少女殉血', 'Animals on the Earth', '四神獣飼殺し', 'ALICE同罪イノセント', '陸と海と空と', '緋ノ糸輪廻ノGEMINI']
ura.query('title in @my_before')

,title,date,initial,kind,r2007,r2008,r2009,r2010,r2011,r2012,...,under50,under100,under200,count_all,count,rank_average,rank_average_exclude_debut,rank_highest,rank_highest_exclude_debut,under100_mean
no,,,,,,,,,,,,,,,,,,,,,
1,桜の花は狂い咲き,1997-04-25,r2005,"[SA, OA, OT]",86.0,126.0,116.0,109.0,96.0,123.0,...,0,4,13,16,13,58.0,46.0,97.0,64.0,0.250000
209,Animals on the Earth,2009-08-26,r2010,[OA],NaN,NaN,NaN,35.0,45.0,76.0,...,2,5,9,10,10,49.0,49.0,48.0,27.0,0.500000
234,四神獣飼殺し,2010-09-26,r2011,[OA],NaN,NaN,NaN,NaN,27.0,71.0,...,1,4,8,9,9,61.0,63.0,38.0,74.0,0.444444
299,陸と海と空と,2015-09-09,r2015,"[OA, CA]",NaN,NaN,NaN,NaN,NaN,NaN,...,1,3,5,5,5,15.0,17.0,26.0,43.0,0.600000
309,緋ノ糸輪廻ノGEMINI,2015-09-09,r2015,[OA],NaN,NaN,NaN,NaN,NaN,NaN,...,1,3,5,5,5,40.0,47.0,47.0,85.0,0.600000


In [476]:
ura.title.values

array(['桜の花は狂い咲き', 'マリーゴールド・ガーデン', '鏡面界 im Juni', 'アンジェ・ノワールの祭戯',
       '紅い睡蓮の午後', '少女忌恋歌', 'パラソルのある風景', '硝子天井のうちゅう', '幻想庭園', 'フラワーチャイルド',
       '恋せよ乙女 -Love story of ZIPANG-', 'ビアンカ', '空宙舞踏会', '木洩れ陽のワルツ',
       '堕ちて候', '月光浴', '楽園喪失 -Paradise Lost-', '舞踏会の手帖', 'エスカルゴ嬉遊曲',
       'サロメティック・ルナティック', '雪のひとひら', 'ダリの宝石店', 'Virtual Fantasy', '小夜啼鳥',
       '星降る夜の天文学 ～BEDSIDE ASTRONOMY～', 'オフェリア遺文',
       "Nous Deux C'est Pour La Vie", '冬物語', '彼と彼女の聖夜', 'ストロベリーパイをお食べ',
       'Only Love Song', 'Wish', '夢のあとに ～après un rêve～', 'エンジェル・エッグの作り方',
       'Chu Chu', 'Sanctuary', '月夜のピエレット', 'Water Drop', 'ぶくぶく空が…',
       'カフェテラスでお茶をどうぞ', "L'eternite", '愛だけじゃない', '日曜日のシエスタ',
       'Cheri Cheri', 'Anniversary of Angel', '天使に寄す', 'LABYRINTH',
       'Halation', '堕落論', 'deep forest', 'LOST CHILD', '月光夜', 'Sacrifice',
       '病める薔薇', 'MALICE', 'à la cuisine', '桂花葬', '閉ざされた画室',
       'un tableau blanc ～絵画旅行～', '闇の翼ですべてをつつむ夜のためのアリア', 'après le noir',
       '宵待草', 'La vie en rose', 'schism', '時の森のソワ

In [477]:
my = ['桜の花は狂い咲き', "Nous Deux C'est Pour La Vie", '月夜のピエレット', '神風', 'Animals on the Earth', '絶國TEMPEST', '四神獣飼殺し', 'La vérité', '真夏の憂愁夫人', '令嬢薔薇図鑑', '陸と海と空と', '緋ノ糸輪廻ノGEMINI']
ura.query('title in @my')

,title,date,initial,kind,r2007,r2008,r2009,r2010,r2011,r2012,...,under50,under100,under200,count_all,count,rank_average,rank_average_exclude_debut,rank_highest,rank_highest_exclude_debut,under100_mean
no,,,,,,,,,,,,,,,,,,,,,
1,桜の花は狂い咲き,1997-04-25,r2005,"[SA, OA, OT]",86.0,126.0,116.0,109.0,96.0,123.0,...,0,4,13,16,13,58.0,46.0,97.0,64.0,0.250000
37,Nous Deux C'est Pour La Vie,1995-10-18,r2005,"[S-2, CA]",44.0,65.0,80.0,118.0,117.0,107.0,...,2,5,14,16,13,60.0,48.0,20.0,26.0,0.312500
49,月夜のピエレット,1997-05-21,r2005,"[OA, OT, S-2, ST, CA]",94.0,172.0,107.0,104.0,92.0,68.0,...,0,3,11,16,13,81.0,68.0,97.0,64.0,0.187500
185,神風,2008-08-27,r2009,"[OA, CA]",NaN,NaN,13.0,21.0,14.0,24.0,...,10,10,11,11,11,1.0,1.0,14.0,4.0,0.909091
209,Animals on the Earth,2009-08-26,r2010,[OA],NaN,NaN,NaN,35.0,45.0,76.0,...,2,5,9,10,10,49.0,49.0,48.0,27.0,0.500000
228,絶國TEMPEST,2010-09-26,r2011,"[OA, CA]",NaN,NaN,NaN,NaN,53.0,69.0,...,0,3,5,9,9,117.0,123.0,73.0,70.0,0.333333
234,四神獣飼殺し,2010-09-26,r2011,[OA],NaN,NaN,NaN,NaN,27.0,71.0,...,1,4,8,9,9,61.0,63.0,38.0,74.0,0.444444
258,La vérité,2012-07-18,r2012,[OA],NaN,NaN,NaN,NaN,NaN,15.0,...,1,5,8,8,8,27.0,27.0,19.0,48.0,0.625000
261,真夏の憂愁夫人,2012-07-18,r2012,"[OA, CA]",NaN,NaN,NaN,NaN,NaN,39.0,...,6,7,8,8,8,2.0,2.0,10.0,1.0,0.875000


In [478]:
# for idx, i in enumerate(map(lambda line: f'<p>{line}</p>', ura_list.title)):
#     print(i, end='')
#     if idx % 50 == 49:
#         print()

In [479]:
# for idx, i in enumerate(map(lambda line: f'<li>{line}</li>', ura_list.title)):
#     print(i, end='')
#     if idx % 100 == 99:
#         print()